In [1]:
from gensim.models import Word2Vec

C:\Users\Harsha\AppData\Local\Programs\Python\Python36-32\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
#from gensim.models import KeyedVectors
#filename = 'GoogleNews-vectors-negative300.bin'
#model = KeyedVectors.load_word2vec_format(filename, binary=True)



from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'intents/glove.6B.100d.txt'
word2vec_output_file = 'intents/glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)



(400000, 100)

In [3]:

from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'intents/glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
word_vec=model.wv
#word_vec.get_vector('cab')

C:\Users\Harsha\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [4]:
x=word_vec.get_vector('pub')
#x.shape


In [5]:
import numpy as np
cab_file=open('intents/cab.dat','r')
stopwords=open('intents/stopwords.txt','r')


stopwrds=[]
for each in stopwords:
    stopwrds.append(each.strip())
#print(stopwrds)
stopwords.close()


In [8]:
import scipy
def eqdist(invec,givenvec):
    d=[]
    for every in invec:
        d.append(np.sqrt(sum(np.square(every - givenvec))))
    #print(d)
    return(min(d))
def cosdist(invec,givenvec):
    d=[]
    for every in invec:
        d.append(scipy.spatial.distance.cosine(every,givenvec))
    #print(d)
    return(min(d))

In [42]:
def file2vec(filename):
    clean_sens=[]
    sens=[]
    file=open(filename,'r')
    for lines in file:
        sens.append(lines.split())
    file.close()
    for i in sens:
        l=[]
        for words in i:
            if '$' in words:
                words=words[1:]
            if words.lower() not in stopwrds and (words.lower() in word_vec.vocab):
                l.append(words.lower())
        clean_sens.append(l)
    #print(clean_sens)
    sens_vec=[]
    for wrds in clean_sens:
        vecs=np.zeros(word_vec.get_vector('pub').shape)
        #print(vecs.shape)
        for wrd in wrds:
            #print(word_vec.get_vector(wrd))
            vecs=vecs+word_vec.get_vector(wrd)
        sens_vec.append(vecs/len(wrds))
    return sens_vec

In [46]:
def getintent(clean_input):
    uinput=clean_input
    givenvec=np.zeros(word_vec.get_vector('pub').shape)
    count=0
    for w in uinput.split():
        if (w.lower() not in stopwrds) and (w.lower() in word_vec.vocab):
            givenvec=givenvec+word_vec.get_vector(w.lower())
            count=count+1
    pubsd=cosdist(file2vec('intents/restaurant.dat'),givenvec)
    cabsd=cosdist(file2vec('intents/cab.dat'),givenvec)
    red=cosdist(file2vec('intents/restart.dat'),givenvec)
    
    #print(pubsd,cabsd,red)
    if pubsd < cabsd :
        if pubsd < red:
            print('restaurant')
            return('restaurant')
        else:
            print('restart')
            return('restart')
    else :
        if cabsd <red:
            print('Cab')
            return('Cab')
        else:
            print('restart')
            return('restart')
#print(givenvec/count)

        

In [50]:
getintent('lets go for a ride')

Cab


C:\Users\Harsha\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


'Cab'